# Układy równań liniowych

### Pojęcia warte poznania

* Układ równań liniowych: https://pl.wikipedia.org/wiki/Układ_równań_liniowych
* Rząd macierzy: https://pl.wikipedia.org/wiki/Rząd_macierzy
* Kombinacja liniowa: https://pl.wikipedia.org/wiki/Kombinacja_liniowa
* Eliminacja Gaussa: https://pl.wikipedia.org/wiki/Metoda_eliminacji_Gaussa, Kincaid-Cheney_*_ str. 245, pełny pseudokod: str. 252
* Pivoting: https://en.wikipedia.org/wiki/Pivot_element#Partial_and_complete_pivoting, K.C. str. 261, pełny pseudokod: str. 267
* Norma wektora: https://pl.wikipedia.org/wiki/Przestrze%C5%84_unormowana, K.C. str. 320
* Norma macierzy: https://pl.wikipedia.org/wiki/Norma_macierzowa
* Macierz dodatnio określona: https://pl.wikipedia.org/wiki/Określoność_formy
* Faktoryzacja LU: https://pl.wikipedia.org/wiki/Metoda_LU, K.C. str. 294
* Faktoryzacja Cholesky'ego: https://pl.wikipedia.org/wiki/Rozkład_Choleskiego, K.C. str. 305

Dodatkowo:
* Wskaźnik uwarunkowania: https://pl.wikipedia.org/wiki/Wska%C5%BAnik_uwarunkowania, K.C. str.321
* Metoda Jacobiego: https://en.wikipedia.org/wiki/Jacobi_method, K.C. 323

Książka dla wytrwałych (naprawdę): Y. Saad "Iterative Methods for Sparse Linear Systems"

In [3]:
from typing import Optional, Tuple
import numpy as np

### Zadanie rozgrzewkowe:
Napisać mnożenie macierzy w ulubionym_\**_ języku programowania.

**Pytanko:** jakie muszą być wymiary mnożonych macierzy? (Który wymiar musi się "zgadzać"?)

**Zadanko:** Uzupełnić brakujące wymiary macierzy w docstringu (z dokładnością do ["alfa-konwersji"](https://pl.wikipedia.org/wiki/Konwersja_α))

In [26]:
def superfast_matrix_multiply(a: np.matrix, b: np.matrix) -> np.matrix:
    n = a.shape[0]
    m = a.shape[1]
    p = b.shape[1]
    c = np.zeros((n,p))
    
    for i in range(0,n):
        for j in range(0,p):
            for k in range(0,m):
                c[i,j] += a[i,k] * b[k,j]
    return c
    pass
m1 = np.matrix([[1, 2],
                [3, 4],
                [4, 5],
                [5, 1]])

m2 = np.matrix([[1, 2, 3],
                [4, 5, 6]])

res = agh_superfast_matrix_multiply(m1, m2)
assert np.allclose(res, m1 * m2), "Wrong multiplication result"

### Zadania
1. Przeczytać rozdz. 7. Kincaida i Cheney'a (Systems of Linear Equations).
2. Przeczytać rozdz. 8. Kincaida i Cheney'a (Additional Topics Concerning Systems of Linear Equations).
3. Napisać kod (w ulubionym_\**_ języku) do eliminacji Gaussa z i bez pivotingu.
4. Rozwiązać poniższy układ równań z pivotingiem i bez, porównać wyniki:

In [27]:
A = np.matrix([[0.0001, -5.0300, 5.8090, 7.8320],
               [2.2660, 1.9950,  1.2120, 8.0080],
               [8.8500, 5.6810,  4.5520, 1.3020],
               [6.7750, -2.253,  2.9080, 3.9700]])

b = np.matrix([9.5740, 7.2190, 5.7300, 6.2910]).transpose()

x = np.linalg.solve(A, b)

**Pytanie**: dlaczego wołamy `transpose()` na wektorze `b`?

Sprawdźmy, czy rozwiązanie jest ok (**Pytanie'**: dlaczego po prostu nie użyjemy `==` lub jakiegoś `equals`?):

In [28]:
np.allclose(np.dot(A, x), b)

True

### Zadania, c.d.

5. Zaimplementować algorytm faktoryzacji LU macierzy
6. (*) Zaimplementować funkcję sprawdzającą, czy dana macierz jest symetryczna i dodatnio określona
7. Zaimplementować algorytm faktoryzacji Cholesky'ego macierzy

In [29]:
def superfast_lu(a: np.matrix) -> Optional[Tuple[np.matrix, np.matrix]]:
    n = a.shape[0]
    l = np.zeros((n,n))
    u = np.zeros((n,n))
    
    for k in range(0,n):
        l[k,k] = 1
        for j in range(k,n):
            sum = 0
            for s in range(0,k):
                sum += l[k,s] * u[s,j]
            u[k,j] = a[k,j] - sum
        for i in range(k+1,n):
            sum = 0
            for s in range(0,k):
                sum += l[i,s] * u[s,k]
            l[i,k] = (a[i,k] - sum) / u[k,k]
    return (l, u)

def superfast_check_spd(a: np.matrix) -> bool:
    l = superfast_cholesky(a)
    llt = superfast_matrix_multiply(l, l.transpose())
    return np.allclose(llt, a)

def superfast_cholesky(a: np.matrix) -> Optional[np.matrix]:
    n = a.shape[0]
    l = np.zeros((n, n))
    
    for k in range(0, n):
        sum = 0
        for s in range(0, k):
            sum += l[k, s] * l[k, s]
        l[k, k] = pow(a[k, k] - sum, 1 / 2)
        for i in range(k+1, n):
            sum = 0
            for s in range(0, k):
                sum += l[i, s] * l[k, s]
            l[i, k] = (a[i, k] - sum) / l[k, k]
    return l

In [30]:
A = np.matrix([[5.0, 3.0, 2.0],
               [ 1.0, 2.0, 0.0],
               [ 3.0, 0.0, 4.0]])

LU = superfast_lu(A)
print(LU[0], " = L")
print(LU[1], " = U")

[[ 1.          0.          0.        ]
 [ 0.2         1.          0.        ]
 [ 0.6        -1.28571429  1.        ]]  = L
[[ 5.          3.          2.        ]
 [ 0.          1.4        -0.4       ]
 [ 0.          0.          2.28571429]]  = U


In [31]:
A = np.matrix([[2.0, -1.0, 0.0],
               [-1.0, 2.0, -1.0],
               [ 0.0, -1.0, 2.0]])

L = superfast_cholesky(A)
print(L, " = L")
print(L.transpose(), " = L^T")

[[ 1.41421356  0.          0.        ]
 [-0.70710678  1.22474487  0.        ]
 [ 0.         -0.81649658  1.15470054]]  = L
[[ 1.41421356 -0.70710678  0.        ]
 [ 0.          1.22474487 -0.81649658]
 [ 0.          0.          1.15470054]]  = L^T


In [32]:
A = np.matrix([[2.0, -1.0, 0.0],
               [-1.0, 2.0,-1.0],
               [ 0.0,-1.0, 2.0]])

superfast_check_spd(A)

True

### Zadania, opcjonalnie
5. zaimplementować metodę Jacobiego (iteracyjne rozwiązywanie układu równań liniowych)
6. za pomocą tejże metody rozwiązać powyższy układ równań

\*  wszystkie referencje odnoszą się do [książki](https://wiki.iiet.pl/lib/exe/fetch.php?media=studia:przedmioty:mownit:numerical_mathematics_and_computing.pdf) David Kincaid, Ward Cheney - "Numerical Mathematics and Computing, 6th edition"
\** _ulubiony_ język programowania staramy się pojmować rozsądnie, tj. z wyłączeniem języków pokroju Prologa, języków z [tej listy](https://en.wikipedia.org/wiki/Esoteric_programming_language) oraz Assemblera i PHP. Haskella można używać na własną odpowiedzialność.